In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import concurrent.futures

In [2]:
#this function extracts the number of pages on Maxpreps for each state.
def get_num_pages(state):
    current_page = 1
    while True:
        url = f'https://www.maxpreps.com/{state}/basketball/21-22/rankings/{current_page}/'
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        next_page_button = soup.find_all('a', class_ = 'btn btn-default')
        button = [butt.text for butt in next_page_button]
        #print(button)
        num = str(current_page + 1)
        if num not in button:
            return int(num)- 1
        current_page += 1

In [3]:
#this function will take in a state as an input and create a pandas dataframe with the school name, state rank, and SOS for each school in the state.
def extract_state_sos(state, current_page):
    url = f'https://www.maxpreps.com/{state}/basketball/21-22/rankings/{current_page}/'
    response = requests.get(url)
    soup = BeautifulSoup(response.text , 'html.parser')
    
    school_elements = soup.find_all('th', class_='school')
    school_elements.pop(0)
    school_names = [school.text for school in school_elements]
    
    school_rk_elements = soup.find_all('td', class_='rank first dw')
    school_rks = [rk.text for rk in school_rk_elements]

    school_sos_elements = soup.find_all('td', class_ = 'strength dw')
    school_sos = [sos.text for sos in school_sos_elements]
    
    school_rating_elements = soup.find_all('td', class_ = 'rating sorted dw')
    school_rating = [sos.text for sos in school_rating_elements]

    return pd.DataFrame({'School': school_names, 'State Rank': school_rks, 'SOS': school_sos, 'Rating': school_rating, 'State': state.upper()})

In [4]:
#function that will remove the city in parenthesis next to the school name.
def remove_city(school):
    return school.split('(')[0].strip()

In [5]:
#function that will loop through all the pages of a state and extract the school name, state rank, and SOS for each school in the state.
def get_state_sos(state):
    state_df = pd.DataFrame()
    for page in range(1, get_num_pages(state) + 1):
        page_data = extract_state_sos(state, page)
        state_df = pd.concat([state_df, page_data], ignore_index = True)
    state_df['School'] = state_df['School'].apply(remove_city)
    return state_df

In [6]:
ak_subset = final_df[final_df['Team 1 State'] == 'AK']
ak_subset.sort_values('Team 1')

NameError: name 'final_df' is not defined

In [9]:
right = get_state_sos('ak')
right

,School,State Rank,SOS,Rating,State
0,Bettye Davis East Anchorage,1,13.4,29.19,AK
1,South Anchorage,2,12.9,23.90,AK
2,Dimond,3,13.6,22.60,AK
3,West Valley,4,4.9,22.25,AK
4,Ninilchik,5,0.0,20.56,AK
...,...,...,...,...,...
109,Deering,110,-5.0,-16.02,AK
110,Hoonah,111,-0.7,-16.23,AK
111,Aniguiin,112,1.8,-16.43,AK
112,Galena,113,2.1,-16.84,AK


In [104]:
#function that will use subset final dataset by state
#then it will create a df of the sos and rating data
#format the school name by deleting parenthesis and the city
#order both alphabetically
#compare length/merge by school name and index?
#after merge is complete, append to a new pandas df with the state name, or use key to put back into original df
right = get_state_sos('ak').sort_values('School')
left = ak_subset.sort_values('Team 1').drop_duplicates(subset = 'Team 1 Label')
left


,Unnamed: 0,Date,Team 1,Team 2,Venue,Game Type,Team 1 Score,Team 2 Score,Outcome,Team 1 Address,...,T2 Free and Reduced Lunch,T2 Pupil/Teacher Ratio,T2 Charter School,T2 Magnet School,T2 Private School,T2 Expenditures/Pupil,T2 School ID,T2 Agency ID,Team 1 Label,Team 2 Label
142964,142964,2/18,Akiachak,Akiak,A,District,69.0,34.0,W,NaN,...,NaN,NaN,0,0,NaN,NaN,NaN,NaN,5615,NaN
142830,142830,2/18,Alak,Nuiqsut Trapper,A,District,81.0,33.0,W,567 Main Street,...,NaN,NaN,0,0,NaN,NaN,NaN,NaN,5606,NaN
142818,142818,1/24,Angoon,Skagway,H,District,49.0,73.0,L,500 Big Dog Salmon Way,...,0.0,10.92,0,0,0.0,25847,20069000310.0,200690.0,5605,5695.0
142940,142940,2/24,Aniak,Newhalen,H,Regular Season,58.0,61.0,L,Aniak #29,...,0.0,10.38,0,0,0.0,58150,20048500199.0,200485.0,5613,5680.0
142918,142918,1/14,Aniguiin,James C,A,Regular Season,42.0,78.0,L,29 Main St.,...,NaN,NaN,0,0,NaN,NaN,NaN,NaN,5612,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144887,144887,2/4,West Anchorage,Eagle River,A,District,74.0,62.0,W,1700 Hillcrest Dr,...,0.0,21.64,0,0,0.0,19178,20018000458.0,200180.0,5714,5634.0
144940,144940,1/27,West Valley,Wasilla,H,Tournament,74.0,36.0,W,3800 Geist Rd,...,0.0,17.17,0,0,0.0,17634,20051000237.0,200510.0,5718,5713.0
144924,144924,2/19,Wrangell,Petersburg,H,Regular Season,30.0,23.0,W,312 Reid St,...,0.0,11.55,0,0,0.0,25238,20066000570.0,200660.0,5717,5688.0
144905,144905,2/9,Yakutat,Hoonah,A,District,61.0,58.0,W,429 Forest Highway,...,0.0,9.46,0,0,0.0,32422,20030000146.0,200300.0,5716,5644.0


### Functions to Extract Strength of Schedule and Ranking

In [ ]:
import os
os.getcwd()
os.chdir('/Users/rdhir/Documents/maxpreps-data-analysis/Data/Final Data (Maxpreps + Demographics')
final_df = pd.read_csv('maxpreps_and_demo.csv', index_col=None)